# PC Lab 6: Regularization Methods 
---

We have already seen linear regression to tackle regression problems. With linear regression, we model a continous outcome as a linear function of the features:

$$\hat{y} = w_{0}x_{0} + w_{1}x_{1} + ... + w_{p}x_{p} = \sum\limits_{i=0}^{p}w_{i}x_{i}.$$

This works well when there are a lot of training observations and when the number of features (the dimensionality of the problem) is not too large. However, there are a couple of situations where ordinary linear regression might give problems:

* When the number of features $p$ becomes large with respect to the number of observations $n$, the variance of the model weights estimated by linear regression increases, which might result in poor predictive performance. Futhermore, there is no longer an analytical solution provided by least squares when $p > n$.
* It is possible that there are a lot of uninteresting or redundant features. If we want a sparse and interpretable model, we might want to do feature selection to reduce $p$.

In this lab, we will cover two solutions to the problems above: subset selection and regularization methods.

## 1. Subset selection methods

In subset selection, we only use a subset of the features that are available. The goal is to come up with a model that is sparse and that generalizes better to unseen data. There are two main strategies for subset selection: in *best subset selection*, we fit all the $p \choose k$ models for each $k = 1, 2, ..., p$ and retain the best model for each $k$. Finally, we select the model that performs best on some measure that controls for overfitting:

![bestsubset](https://raw.githubusercontent.com/gdewael/teaching/main/predmod/lab6/img/best_subset.png)

This becomes quickly unfeasible for large values of $p$. Therefore, an alternative approach is to perform *stepwise selection*, which explores a much smaller set of feature combinations. Stepwise selection can be performed either backward or forward. 

![forward](https://raw.githubusercontent.com/gdewael/teaching/main/predmod/lab6/img/forward.png)

Finally, it is important to account for the fact that the MSE (or, equivalently, the RSS) will always go down on the training data as we add more and more features. To select the best model out of several candidates, it is important to have an estimate of the test error of each model. This can be done indirectly by using a metric that penalizes for model complexity such as the AIC or the adjusted $R^2$. Another option is to use cross-validation to get an estimate of the test error.

**Let's apply subset selection on two datasets.** The first dataset contains features of mixtures used to produce concrete. The goal is to predict the compressive strength of the concrete.

In [ ]:
!wget https://raw.githubusercontent.com/tfmortie/mlcourse/refs/heads/main/06_regularization/concreteComprStrength.txt
!wget https://raw.githubusercontent.com/tfmortie/mlcourse/refs/heads/main/06_regularization/fc_data_new.csv
!wget https://raw.githubusercontent.com/tfmortie/mlcourse/refs/heads/main/06_regularization/communities.csv

In [ ]:
# Read in the data
import pandas as pd

concretedata = pd.read_table('concreteComprStrength.txt', delim_whitespace=True, header=0, index_col=None)
# Extract features and label for later usage
X = concretedata.drop(['compressiveStrength'], axis=1)
y = concretedata['compressiveStrength']

print(concretedata.shape)
concretedata.head()

Let's build up an intuition for the code we need for doing best subset selection with linear regression. We will implement Algorithm 6.1 as shown above. First we split the data and look at how we can generate all possible combinations of features:

In [ ]:
from itertools import chain, combinations

list_of_features = ["ft.1", "ft.2", "ft.3"]
for subset in chain.from_iterable(combinations(list_of_features, r) for r in range(1, len(list_of_features)+1)):
    print(subset)

Now we can use this code to iterate not through `list_of_features` but through the possible features. For every subset we fit a model and keep it in a dictionary that records the score for that specific subset of features:

In [ ]:
from sklearn.linear_model import LinearRegression
from itertools import chain, combinations

list_of_features = X.columns
scoring_dict = {}

for subset in chain.from_iterable(combinations(list_of_features, r) for r in range(1, len(list_of_features)+1)):
    X_subset = X[list(subset)]
    model = LinearRegression()
    model.fit(X_subset, y)
    scoring_dict[subset] = model.score(X_subset, y)

Taking five random samples from this dict to get an idea what's in it:

In [ ]:
import random
random.sample(list(scoring_dict.items()), 5)

Getting the best model:

In [ ]:
max(scoring_dict, key=scoring_dict.get), scoring_dict[max(scoring_dict, key=scoring_dict.get)]

We see that the best model (R² score-wise) is the one where all features are present. However, this is estimated on training data, so it might very well be that this result is due to overfitting. To do this in a better way, we will use cross-validation, as seen in the previous practical. The following code block expands upon the previous code block with these things in mind: **This code may take a little while to run.**

In [ ]:
from sklearn.model_selection import KFold, cross_validate
import numpy as np


list_of_features = X.columns
scoring_dict = {}

#NEW:
# Define splitter object outside loop to ensure same splits over loop
splitter = KFold(n_splits=5, shuffle=True, random_state=None)
####

for subset in chain.from_iterable(combinations(list_of_features, r) for r in range(1, len(list_of_features)+1)):
    X_subset = X[list(subset)]
    model = LinearRegression()

    # NEW: instead of simply fitting the model on all train data,
    # now we use cross validation to fit and evaluate model
    # cross_validate() returns a dict with key 'test_score' containing
    # a list of  CV performances of every split
    scores = cross_validate(model, X_subset.values, y.values, cv=splitter)
    scoring_dict[subset] = np.mean(scores['test_score'])

Let's visualize how the test performance changes in function of how many features we had in the model:

In [ ]:
import matplotlib.pyplot as plt

plt.scatter([len(k) for k in scoring_dict.keys()], list(scoring_dict.values()), alpha = 0.25)
plt.xlabel('number of features')
plt.ylabel('R² score')
# taking the best model for every 'n' number of features and plotting them separately:
best_score_n = [
    max([v for k, v in scoring_dict.items() if len(k) == i])
    for i in range(1, X.shape[1]+1)
]
plt.scatter(range(1, X.shape[1]+1), best_score_n)
print('Best feature combination:', max(scoring_dict, key=scoring_dict.get))

We see that one certain combination of 5 features already results in a performance on +- the same level as the one with all features. This finding begs the question: could we have found this combination more efficiently, without testing out all possible feature combinations?

Now, we will introduce a second dataset. This time, the features are measurements from a flow cytometry experiment. The 'SC' features measure scatter, and say something about the morphologhy of the cells (FSC: forwad scatter, SSC: sideway scatter). The 'FL' features are fluorescence features from different parts of the spectrum. There are two possible bacterial species present in the dataset. The goal is to classify the correct species based on the measurements from the flow cytometer. Species number one corresponds to *Pseudomonas putida*, while species number 6 is *Brachybacterium faecium*. We will use logistic regression to do the classification.

This dataset has 10 features instead of the 8 features encountered in the previous dataset. We could repeat the same *best subset selection* method, but this time we will implement *stepwise forward selection*.

In [ ]:
# Read in the data
bacterialdata = pd.read_csv('fc_data_new.csv', index_col=0)
bacterialdata = bacterialdata.drop(['Width', 'Time'], axis=1).reset_index(drop=True)

In [ ]:
bacterialdata.head()

<div class="alert alert-success">

<b>EXERCISE 1.1</b>: **Complete the code below that implements forward stepwise selection.**
</div>  

In [ ]:
from sklearn.linear_model import LogisticRegression

X = bacterialdata.drop(['species'], axis=1)
y = bacterialdata['species']
scoring_dict = {}
splitter = KFold(n_splits=5, shuffle=True, random_state=None)


features_already_in_model = set()
features_not_in_model = set(X.columns)

# Implement code here 

In [ ]:
for k, v in scoring_dict.items():
    print(k, v)

## 2. Regularization methods: ridge and lasso regression

Ridge and lasso regression are regularization techniques commonly employed in regression analysis to address the issue of overfitting. Overfitting occurs when a model learns the training data too well, capturing noise and leading to poor generalization on unseen data. Ridge regression adds a regularization term to the cost function (i.e. MSE), which penalizes large coefficients, encouraging the model to prioritize a simpler, smoother solution:

$$\sum_{i=1}^{n}\Big(y_{i}-\sum_{j=0}^{p}w_{j}x_{ij}\Big)^{2}\quad\text{subject to}\quad\sum_{j=1}^{p}w_{j}^{2}=s.$$

Lasso regression, on the other hand, uses a similar regularization approach but penalizes the absolute values of the coefficients: 

$$\sum_{i=1}^{n}\Big(y_{i}-\sum_{j=0}^{p}w_{j}x_{ij}\Big)^{2}\quad\text{subject to}\quad\sum_{j=1}^{p}|w_{j}|=s.$$

The key distinction is that lasso regression can also induce sparsity, effectively setting some coefficients to zero, leading to feature selection. By adding these regularization terms, ridge and lasso regression help prevent overfitting by constraining the model's complexity, making it more robust and improving its performance on new, unseen data.

### Linear models for high dimensional data

We will apply ridge regression on the Communities and Crime Data Set. The dataset contains 123 population statistics on 1994 communities. We would like to predict the number of violent crimes per 100000 inhabitants. This is the final column of the dataframe. Of the 123 features, a lot contain missing values, so we will drop these columns and use only 99 features:

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('./communities.csv')

# Drop columns with missing values
data = data.dropna(axis=1)
print(data.shape)
data.head()

<div class="alert alert-success">

<b>EXERCISE 2.1</b>: **Use linear regression and ridge regression to predict the number of violent crimes per 100,000 inhabitants. Use 5-fold cross-validation to evaluate both models. The [_RidgeCV_](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) class in Scikit-learn automatically performs cross-validation to tune the hyperparameter that determines the amount of regularization, so you don't need to implement a second cross-validation loop. Which model performs best?**
</div>  

**Before we apply ridge regression, it's important to make sure that all the features are on the same scale. If one of the features is on a completely different scale (let's say, income can be measured in dollars or in thousands of dollars), this might lead the ridge regression coefficient to change substantially because of the penalty term in the optimization problem. We can make sure that all the features are on the same scale by means of standardization:**

$$\tilde{x}_{ij} = \frac{x_{ij}-\bar{x}_{j}}{\sqrt{\frac{1}{n}\sum_{i=1}^{n}(x_{ij} - \bar{x}_{j})^2}}.$$

We can do this with the [_StandardScaler_](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class in Scikit-learn. We will do the scaling each time in the cross-validation loop using only training data statistics (*do you still remember why we only use training statistics to do this?*).

In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# Select X and y
y = data['ViolentCrimesPerPop'].values
X = data.drop(['ViolentCrimesPerPop'], axis=1).values

kf = KFold(n_splits=5)

linreg_scores = []
ridge_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize the scaler
    scaler = StandardScaler()

    # scale X


    # initialize the models


    # fit, and store the accuracy on the test set

print('Average validation fold R² of linear regression: {}'.format(np.mean(linreg_scores)))
print('Average validation fold R² of ridge regression: {}'.format(np.mean(ridge_scores)))

Now suppose that we don't have 99 features, but four times as many features. And suppose that a lot of features are correlated. This situation is very common in lots of datasets. We will mimic this situation by adding correlated features to our original feature matrix:

In [ ]:
X_1 = X + np.random.normal(loc=0, scale=0.05, size=(X.shape))
X_2 = X + np.random.normal(loc=0, scale=0.1, size=(X.shape))
X_3 = X + np.random.normal(loc=0, scale=0.01, size=(X.shape))
X_expanded = np.concatenate((X, X_1, X_2, X_3), axis=1)

X_expanded.shape

<div class="alert alert-success">

<b>EXERCISE 2.2</b>: **Repeat the comparison between ridge regression and linear regression from above, but with the new feature matrix that contains correlated features. What happens with the performance of linear regression? What happens with the regularization parameter?**
</div>  

### Lasso regression and feature selection

Suppose that we are interested in selecting only a couple of features out of a high dimensional dataset. Let's fit ridge regression and lasso regression on the data and look at the model coefficients for both models:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

ridgemodel = RidgeCV(cv=5)
lassomodel = LassoCV(cv=5, max_iter=10000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ridgemodel.fit(X_train, y_train)
ridgescore = np.round(ridgemodel.score(X_test, y_test),2)
lassomodel.fit(X_train, y_train)
lassoscore = np.round(lassomodel.score(X_test, y_test),2)

# Plot of the coefficients for ridge regression
fig, ((ax1, ax2)) = plt.subplots(figsize=(15, 10), nrows=2)
pd.Series(ridgemodel.coef_).plot(kind='bar', ax=ax1)
ax1.set_title('Ridge regression coefficients. Test set R²: {}'.format(ridgescore)).set_fontsize(40)
ax1.get_xaxis().set_ticks([])
ax1.set_xlabel('Features')

# Plot of the coefficients for the lasso
pd.Series(lassomodel.coef_).plot(kind='bar', ax=ax2)
ax2.set_title('Lasso regression coefficients. Test set R²: {}'.format(lassoscore)).set_fontsize(40)
ax2.get_xaxis().set_ticks([])
ax2.set_xlabel('Features');

The lasso applies regularization by constraining the sum of the absolute values of the model coefficients (the L1-norm). A result of this is that a lot of model coefficients are set to zero: the lasso performs **feature selection**. This is not the case for ridge regression: the model weights are rarely set to zero. Feature selection is a nice property if we want an interpretable model. Let's list the features with non-zero coefficients in the lasso:

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

fig, ax = plt.subplots(figsize=(5,9))
sns.barplot(x= pd.Series(lassomodel.coef_[lassomodel.coef_ != 0]),
            y=data.drop(['ViolentCrimesPerPop'], axis=1).columns[lassomodel.coef_ != 0],
            palette=['steelblue' if n > 0 else 'coral' for n in lassomodel.coef_[lassomodel.coef_ != 0]],
            ax=ax);
ax.set_title('Lasso model coefficients for the Community Crime dataset').set_fontsize(20);